## Análise de sentimentos

### Importando as bibliotecas necessárias para o projeto

In [1]:
import pandas as pd
import emoji
import spacy
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.svm import LinearSVC

### Carregando o spaCy em português

In [2]:
nlp = spacy.load('pt_core_news_lg')

### Lendo a base de dados diretamente do repositório do Github

In [3]:
df = pd.read_csv(
    'https://github.com/guilhermenoronha/sentiment_analysis_chapter/raw/main/dataset/sentiment_analysis.zip', 
    sep=',',
    index_col=[0]
)
pd.set_option('display.max_colwidth', None)

### Verificando a quantidade de linhas e colunas da base de dados

In [4]:
df.shape

(49459, 3)

### Verificando a estrutura da base por meio de uma amostra simples

In [5]:
df.head()

,id,text_pt,sentiment
0,1,"Mais uma vez, o Sr. Costner arrumou um filme por muito mais tempo do que o necessário. Além das terríveis seqüências de resgate no mar, das quais há muito poucas, eu simplesmente não me importei com nenhum dos personagens. A maioria de nós tem fantasmas no armário, e o personagem Costers é realizado logo no início, e depois esquecido até muito mais tarde, quando eu não me importava. O personagem com o qual deveríamos nos importar é muito arrogante e superconfiante, Ashton Kutcher. O problema é que ele sai como um garoto que pensa que é melhor do que qualquer outra pessoa ao seu redor e não mostra sinais de um armário desordenado. Seu único obstáculo parece estar vencendo Costner. Finalmente, quando estamos bem além do meio do caminho, Costner nos conta sobre os fantasmas dos Kutchers. Somos informados de por que Kutcher é levado a ser o melhor sem pressentimentos ou presságios anteriores. Nenhuma mágica aqui, era tudo que eu podia fazer para não desligar uma hora.",negativo
1,2,"Este é um exemplo do motivo pelo qual a maioria dos filmes de ação são os mesmos. Genérico e chato, não há nada que valha a pena assistir aqui. Um completo desperdício dos talentos de Ice-T e Cubo de Gelo que foram mal aproveitados, cada um comprovando que são capazes de atuar e agir bem. Não se incomode com este, vá ver New Jack City, Ricochet ou assistir New York Undercover para Ice-T, ou Boyz no Hood, Higher Learning ou Friday for Ice Cube e ver o negócio real. Ice-Ts horrivelmente clichê diálogo sozinho faz este filme ralar os dentes, e eu ainda estou me perguntando o que diabos Bill Paxton estava fazendo neste filme? E por que diabos ele sempre interpreta exatamente o mesmo personagem? Dos extraterrestres em diante, todos os filmes que eu vi com Bill Paxton o fizeram interpretar exatamente o mesmo personagem irritante, e pelo menos em Aliens seu personagem morreu, o que o tornou um pouco gratificante ... No geral, esse é lixo de ação de segunda classe. Existem incontáveis ​​filmes melhores para ver, e se você realmente quiser ver esse filme, assista a Judgment Night, que é praticamente uma cópia carbono, mas tem melhor atuação e um roteiro melhor. A única coisa que fez isso valer a pena assistir foi uma mão decente na câmera - a cinematografia era quase refrescante, o que chega perto de compensar o horrível filme em si - mas não é bem assim. 4/10",negativo
2,3,"Primeiro de tudo eu odeio esses raps imbecis, que não poderiam agir se tivessem uma arma pressionada contra suas testas. Tudo o que eles fazem é amaldiçoar e atirar um no outro e agir como uma versão clichê de gangsters. O filme não leva mais de cinco minutos para explicar o que está acontecendo antes que já estivessem no armazém. Não há um único personagem simpático nesse filme, com exceção do sem-teto, que também é o único com metade do cérebro. William Paxton e William Sadler são ambos ""hill billies"" e Sadler é tão vilão quanto os gângsteres. Eu não gostava dele desde o começo. O filme está cheio de violência sem sentido e especialidade de Walter Hills: pessoas caindo de janelas com vidros voando por toda parte. Não há praticamente nenhum enredo e é um grande problema quando você torce por ninguém. Todo mundo morre, exceto Paxton e o sem-teto e todos recebem o que merecem. Os dois únicos negros que podem atuar são o sem-teto e o viciado, mas são atores de profissão, não irritantes rappers feios. Fique longe dessa porcaria. e observe 48 horas 1 e 2 em vez disso. No mínimo, eles têm personagens de que você gosta, senso de humor e nada além de atores reais no elenco.",negativo
3,4,"Nem mesmo os Beatles puderam escrever músicas que todos gostassem, e embora Walter Hill não seja um mop-top, ele é incomparável quando se trata de filmes de ação instigantes. Os anos noventa chegaram e as plataformas sociais estavam mudando em música e cinema, o surgimento da estrela de cinema do Rapper estava em pleno andamento, a atuação ficou em segundo plano para cada homem domina

### Deletando colunas que não são de interesse

In [6]:
df.drop(columns=['id'], inplace=True)

### Identificando quais são os sentimentos na base

In [7]:
df['sentiment'].drop_duplicates()

0        negativo
12389    positivo
Name: sentiment, dtype: object

### Verificando a proporção de sentimentos para cada um

In [8]:
df['sentiment'].value_counts()

sentiment
negativo    24765
positivo    24694
Name: count, dtype: int64

### Balanceando a base de acordo com os sentimentos

In [9]:
min_rows = df.groupby('sentiment').apply(lambda x: len(x)).min()
df = df.groupby('sentiment').apply(lambda x: x.sample(min_rows)).reset_index(drop=True)
df['sentiment'].value_counts()

sentiment
negativo    24694
positivo    24694
Name: count, dtype: int64

### Transformando a base de dados

In [10]:
def clean_text(sentence):
    doc = nlp(sentence) 
    tokens = [token.lemma_.lower() for token in doc 
              if  not token.is_punct and # Filter punctuation 
                  not token.is_stop  and # Filter stopwords
                  not token.like_num and # Filter numeric representations
                  not token.like_url     # Filter urls
             ]
    cleaned_text = emoji.replace_emoji(' '.join(tokens), replace='') # Remove emoticons
    return cleaned_text.replace("  ", " ") # Remove extra whitespaces
df['processed_text'] = df['text_pt'].apply(clean_text)

### Comparando os textos original e transformado

In [11]:
df.drop(columns=['sentiment']).head(1)

,text_pt,processed_text
0,"Este filme foi bom relativamente durante as primeiras partes dele. Nós temos uma história, a partir de 3 pontos de vista. Então vamos encontrar algumas pistas e completar a história. Oh, espere ... nada disso importa, porque os caras do FBI são os bandidos! Embora isso tenha sido uma grande reviravolta ... foi quase uma reviravolta terrível. Eu imediatamente rebaixei o filme de um 7 talvez 8 para 3 com base nos últimos 10-15 minutos dele. Alguém mais não vê porque o twist é tão ruim? Sim, é um bom choque. Mas é ruim porque não tem absolutamente nada a ver com a hora e os vinte minutos anteriores. Não há conexão com os assassinos. Os assassinos estão em cerca de 5 minutos desse filme como assassinos e os dois agentes do FBI estão em apenas 15 minutos da hora anterior e vinte. Nós conseguimos ... vigilância ... Oh, os assassinos são voyeurs. O que não faz sentido, porque eles foram descritos apenas em termos limitados como sendo apenas psicopatas. E as horas e vinte minutos de vigilância que estamos vendo nos 3 andares saem pela janela enquanto todos estão mortos em 5 minutos. Tudo isso torna o final ainda mais ridículo. Oh, eles mataram um monte de agentes do FBI no começo ... o que agentes do FBI dormem juntos? Tudo na mesma sala. Ser encontrado e assassinado por amadores e depois personificado por pessoas que não sabem nada sobre serem agentes do FBI? Um policial a 3 metros de distância, aparentemente, não consegue acertar nenhum com uma pistola policial padrão que pode disparar vários tiros. Eu odeio filmes que tentam fazer você se sentir como isso poderia ser real quando eles fazem saltos absurdos que eles acham que vamos acreditar. A outra coisa é que o filme termina cerca de 10-15 minutos depois que eles são revelados como os assassinos com uma garota de pé no campo em algum lugar ...",filme relativamente durante primeiro parte de ele história vista ir encontrar pista completar história oh esperar importar cara fbi bandido ter ser reviravolta quase reviravolta terrível imediatamente rebaixei filme base último 10-15 minuto de ele alguém ver twist ruim choque ruim absolutamente hora minuto anterior haver conexão assassino assassino cerca minuto filme assassino agente fbi minuto hora anterior conseguir vigilância oh assassino voyeurs sentido descritos termos limitar ser psicopata hora minuto vigilância estar ver andar sair janela morto minuto tornar ridículo oh matar monte agente fbi começo agente fbi dormer junto mesmo sala encontrar assassinar amador personificar pessoa saber ser agente fbi policial metro distância aparentemente conseguir acertar nenhum pistola policial padrão disparar tiro odeio filme tentar sentir poder real salto absurdo achar ir acreditar filme terminar cerca 10-15 minuto revelar assassino garota pé campo algum


### Removendo a coluna original, pois não será mais usada

In [12]:
df.drop(columns=['text_pt'], inplace=True)

### Preparando o LinearSVC e separando a base de dados em bases de treinamento e teste

In [13]:
tfidf = TfidfVectorizer()
svm = LinearSVC(dual='auto')
steps = [('tfidf',tfidf),('svm',svm)]
pipe = Pipeline(steps)
X = df['processed_text']
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
print(X_train.shape,y_test.shape)

(39510,) (9878,)


### Preparando o pipeline de aprendizado

In [14]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('svm', LinearSVC(dual='auto'))])

### Testando o modelo

In [15]:
y_pred = pipe.predict(X_test)

### Obtendo as métricas de avaliação

In [16]:
print('Métricas')
print(classification_report(y_test,y_pred))
print('Matriz de Confusão')
cmtx = pd.DataFrame(
    confusion_matrix(y_test, y_pred, labels=['positivo', 'negativo']), 
    index=['true:positivo', 'true:negativo'], 
    columns=['pred:positivo', 'pred:negativo']
)
print(cmtx)

Métricas
              precision    recall  f1-score   support

    negativo       0.89      0.87      0.88      4966
    positivo       0.88      0.89      0.88      4912

    accuracy                           0.88      9878
   macro avg       0.88      0.88      0.88      9878
weighted avg       0.88      0.88      0.88      9878

Matriz de Confusão
               pred:positivo  pred:negativo
true:positivo           4377            535
true:negativo            624           4342


### Testando o modelo com novas frases

In [17]:
sentence_1 = 'Achei esse filme muito bom'
sentence_2 = 'Perdi duas horas da minha vida'
print(f'Sentence: {sentence_1}\nSentiment:{pipe.predict([sentence_1])[0]}')
print(f'Sentence: {sentence_2}\nSentiment:{pipe.predict([sentence_2])[0]}')

Sentence: Achei esse filme muito bom
Sentiment:negativo
Sentence: Perdi duas horas da minha vida
Sentiment:negativo
